In [5]:
import pandas as pd
from Bio import Entrez, SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.SeqRecord import SeqRecord

# Ustaw swój adres email tutaj, oraz klucz API NCBI, jeśli go posiadasz
Entrez.email = "maksnecki@gmail.com"
#Entrez.api_key = "your_ncbi_api_key"  # Umieść tutaj swój klucz API NCBI, jeśli masz

# Wczytaj dane z pliku CSV
data = pd.read_csv('C:/Users/maksn/Downloads/ready_data_final_ver2.csv', sep=",", header=0)

def fetch_genome_sequence(genome_id):
    handle = Entrez.efetch(db="nucleotide", id=genome_id, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    return record


In [6]:
unique_clusters = data['__mclCluster'].unique()

# Sortowanie unikalnych wartości klastrów, jeśli to konieczne
unique_clusters.sort()

In [14]:
unique_clusters

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123], dtype=int64)

In [7]:
# Iteracja przez numery klastrów od 1 do 123
for cluster_number in unique_clusters:
    # Filtruj dane dla danego klastra
    cluster_data = data[data['__mclCluster'] == cluster_number]

    # Iteracja przez unikalne genome_id w klastrze
    for genome_id in cluster_data['genome_id'].unique():
        # Pobierz rekord sekwencji genomu z NCBI
        try:
            genome_record = fetch_genome_sequence(genome_id)
        except Exception as e:
            print(f"Nie udało się pobrać sekwencji dla {genome_id}: {e}")
            continue
        
        # Filtruj dane dla danego genome_id w klastrze
        genome_data = cluster_data[cluster_data['genome_id'] == genome_id]
        
        # Utwórz obiekt SeqRecord z pobranej sekwencji
        record = SeqRecord(genome_record.seq, id=genome_record.id, name=genome_record.name, description=genome_record.description)
        
        # Dodaj adnotacje
        record.annotations["molecule_type"] = "DNA"
        
        # Dodaj domeny jako cechy do rekordu
        for index, row in genome_data.iterrows():
            feature = SeqFeature(
                location=FeatureLocation(start=row['domain_start'], end=row['domain_end'], strand=row['strand']),
                type="CDS",
                qualifiers={
                    "domain_id": row['t_id'],
                    "domain_name": row['t_name'],
                    "domain_length": str(row['domain_length'])
                }
            )
            record.features.append(feature)
        
        # Zapisz rekord do pliku GenBank
        output_file = f"{genome_id}_cluster_{cluster_number}_domains.gb"
        with open(output_file, "w") as output_handle:
            SeqIO.write(record, output_handle, "genbank")
        
        print(f"Plik GenBank zapisany jako: {output_file}")

Plik GenBank zapisany jako: NC_009541.1_cluster_1_domains.gb
Plik GenBank zapisany jako: NC_015453.1_cluster_1_domains.gb
Plik GenBank zapisany jako: NC_015454.1_cluster_1_domains.gb
Plik GenBank zapisany jako: NC_018834.1_cluster_1_domains.gb
Plik GenBank zapisany jako: NC_018838.1_cluster_1_domains.gb
Plik GenBank zapisany jako: NC_018839.1_cluster_1_domains.gb
Plik GenBank zapisany jako: NC_018840.1_cluster_1_domains.gb
Plik GenBank zapisany jako: NC_018841.1_cluster_1_domains.gb
Plik GenBank zapisany jako: NC_018842.1_cluster_1_domains.gb
Plik GenBank zapisany jako: NC_018845.1_cluster_1_domains.gb
Plik GenBank zapisany jako: NC_018847.1_cluster_1_domains.gb
Plik GenBank zapisany jako: NC_018849.1_cluster_1_domains.gb
Plik GenBank zapisany jako: NC_018851.1_cluster_1_domains.gb
Plik GenBank zapisany jako: NC_018852.1_cluster_1_domains.gb
Plik GenBank zapisany jako: NC_022334.1_cluster_1_domains.gb
Plik GenBank zapisany jako: NC_022335.1_cluster_1_domains.gb
Plik GenBank zapisany ja

In [8]:
import os
import shutil

In [9]:
# Ścieżka do folderu z plikami
source_folder = 'C:/Users/maksn/Downloads'
# Ścieżka do nowego głównego folderu
destination_folder = 'C:/Users/maksn/Downloads/proj_domain_clusters'

# Upewnij się, że folder docelowy istnieje
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Iteracja po plikach w folderze źródłowym
for filename in os.listdir(source_folder):
    # Sprawdzenie, czy plik ma odpowiedni format
    if '_cluster_' in filename and '_domains' in filename:
        # Wydobycie numeru klastra
        cluster_number = filename.split('_cluster_')[1].split('_domains')[0]
        cluster_folder_name = f'cluster_{cluster_number}'
        
        # Ścieżka do folderu klastra
        cluster_folder_path = os.path.join(destination_folder, cluster_folder_name)
        
        # Upewnij się, że folder klastra istnieje
        if not os.path.exists(cluster_folder_path):
            os.makedirs(cluster_folder_path)
        
        # Ścieżki do pliku źródłowego i docelowego
        source_file_path = os.path.join(source_folder, filename)
        destination_file_path = os.path.join(cluster_folder_path, filename)
        
        # Przeniesienie pliku
        shutil.move(source_file_path, destination_file_path)

print("Pliki zostały przeniesione.")

Pliki zostały przeniesione.
